#|hide notes

## Developer notes

This is a first attempt at using nbdev to develop a package, commentary is added to the code to explain the process. [nbdev](https://github.com/fastai/nbdev) is a tool to develop packages using python jupyter notebooks. It provides a set of tools (can check via `nbdev_help`) which makes what you write in this notebook a package. If you follow conventions it uses it also includes testing which can be automatically turned into github actions, and releases for both conda and pypi. It accomplishes this by transforming notebook code into a standard python package and using information in certain files and folders (notably `settings.ini` and `*.ipynb`) to create a package.

I think it's important to organize other files you add to this structure so you're aware of which files are auto generated and which ones aren't. This also encourages a different way to approach the code that is focused on explanation more so than pure structure. For examply when creating the class is this notebook I purposefully didn't put many methods with the intiail declaration. Instead I included the basics and then used `fastcore.utils.patch` to "patch" in additional methods. These methods are then associated with the class (I believe it uses the typeclass of self to know where it goes) when converted. In terms of final code it looks a little less organized but in terms of process you can create your object, add some tests to ensure it works, then patch in a function, test that inidividual function, and continue till all functions are added. This goes better with explanations but worse for at a glance results. Because of this library imports are not all at the top of the notebook. Instead they are added as needed as the code is written for understanding. This may cause some issues with remembering to include packages in the `conda.env.yml`.

A plus from this is docs are also auto created. This means each section should be demarked by a `#|hide` or `#|export` tag, with the `#|export` tag being used for all code that makes it to production. It appears `#|hide` can also be applied to markdown to exclude it, having `#|export` in markdown doesn't force include it though, so more likely an interaction with any tags. If you use neither it appears in the docs but not in the production code. I should also mention quickly that it appears `nbdev` is still relatively new but it feels like you should be able to make any kind of python package with this, including CLI. Also note that `#|export` which needed for including to modules does not include in documentation always, this looks like it applies to all non-function exports. All code blocks seen are either from functions or are not exported.

I believe the tests that go to github actions are ones using the `fastcore.test` library. So using this should offer that functionality.

As a more minor note I've developed in [VSCode](https://code.visualstudio.com/) using a jupyter notebook environment built from `conda.env.yml` and pointing my Jupyter Server to the associated `python` created (`./venv/bin/python`). I find this makes most things easier but some integration (certain auto-completions) of `nbdev` is currently designed to work through a server based [jupyter notebook](https://jupyter.org/).

Other sections of markdown labeled with `#|hide` should be assumed as further developer notes and not intended for the final product.

#|hide

## References for development

1. [nbdev_cards](https://github.com/fastai/nbdev_cards), an example they use in their [video tutorial](https://www.youtube.com/watch?v=l7zS8Ld4_iA)

#|hide

This bit of code indicates which python file (in this case `pingme` it should create within the package. Each notebook should export to a different file).

In [ ]:
#| default_exp core

#|hide

The following folders/files are included outside of the nbdev framework, I don't want to modify nbdev autogenerated files of course.
- `conda.env.yml` - conda environment file for development
- `./config/`
    - `config.env` - environment variables
- `./output` - folder for output files
- `./cards`
    - `./cards/default_card.json` - default card for the pingme app

And within the framework I've modified
- `*.ipynb` - notebooks, the prepending ##_ indicates the order they should be run in, but does not affect package naming
    - `index.ipynb` is special as it generates the `README.md` file
- `settings.ini` - configuration file for nbdev

Other files were autogenerated by nbdev and the majority are not intended to be modified.


# PingMe

To install notebook development environment run the following command:
`conda create env -p ./.venv --file conda.env.yml`

NOTE: this may not be the nbdev approach but leaving for now, pretty sure I use pip for envs.

#|hide

I thought of `#|hide` as dev mode and `#|export` as production. This means that if you don't `#|export` what you need it may not work and that you have to identify what is for development only and `#hide` it. This can cause some issues as libraries in `#|hide` may be used in dev and not work in production. This is just something to be aware of. The point of also keeping it in dev mode is to minimize the bloat for production.

In [ ]:
#|hide
# Development libraries, not required for package.
from nbdev.showdoc import *
import fastcore.test # For running tests

In [ ]:
#|export
# Inlcuded libraries, other libraries are included with the methods that use them.
# FIXME: Currently this is not showing in the docs, currently it seems no librs are
from fastcore.utils import patch # decorator to patch in new methods to a class

#|hide

I'm doing project based development for paths. I have an expectation that all paths I use in the program should be relative to the current project folder and if not that those values are provided via a config or args. No information about the system paths should be available and thus the code base should also be safe for public repositories as well. We don't want to expose system structure. To handle this the variables used are set in a environmental variable that points to a file which is in `./gitignore`. All env variables for this notebook are prepended with the program name `PINGME_`. This is to avoid potential conflicts with other programs. I assume a default `config.env` when looking though automatically.

In [ ]:
#|hide
# For development sets real config values which are not stored in git
# Check if config.env is present
import os
from pathlib import Path # for type hinting and file checking

if os.environ.get("PINGME_CONFIG_PATH") is None and Path("./config/config.env").is_file():
    os.environ["PINGME_CONFIG_PATH"] = "./config/config.env"

## Set ENV variables from file
Used for running the notebook in dev mode, not used in sections with `#|export`.

#|hide

The way variables are handled is 2 ways, one is on a per card basis and the other is general settings. The card is a yaml format and contains variables which can be filed in from both a context and from ENV variables. To handle this we use load_dotenv, which does not override values on default. We will call it multiple times which means values set on ENV will have priority, then we'll call it on the specific config, then default values. This will obtain all the ENV vars we need. The card.yaml will then be loaded and ENV variables replaced from the loaded ENV.

In [ ]:
#|export
from dotenv import load_dotenv # for loading config from .env files

def set_env_variables(config_path) -> bool:
    # Order of precedence: environment variables > .env file > default values

    # Set the env vars first, this is needed for the card.yaml to replace ENV variables
    if config_path is not None:
        load_dotenv(config_path)
    load_dotenv("./config/config.default.env")

    return True


In [ ]:
#|export
import os
set_env_variables(os.environ["PINGME_CONFIG_PATH"])

#|hide check env vars.

In [ ]:
#|hide
for k, v in os.environ.items():
    # If ENV var starts with PINGME_ then print
    if k.startswith("PINGME_"):
        print(f"{k}={v}")

## Notebook Variables

Here you can see included potential variables and their ENV variable names.

#|hide

Values here are not used in this package, they're only for notebook.

In [ ]:
#|hide
import json

PINGME_ENV_CARD_NAME = os.environ.get('PINGME_ENV_CARD_NAME',None)
PINGME_ENV_CARD_CONTEXT = json.loads(os.environ.get('PINGME_ENV_CARD_CONTEXT', '{}'))
PINGME_ENV_CARD_DIR = os.environ.get('PINGME_ENV_CARD_DIR', None)
PINGME_ENV_CARD_EXT = os.environ.get('PINGME_ENV_CARD_EXT', None)

# For reducing spam when doing dev work
SEND_EMAIL = True
SEND_WEBHOOK = True
SEND_LOGFILE = True

## PingMe class
> Is used to send notifications to a webhook (designed for use with [incoming webhooks](https://docs.microsoft.com/en-us/microsoftteams/platform/webhooks-and-connectors/how-to/add-incoming-webhook in Microsoft Teams)), e-mail, and/or a log file. The class stores a json payload which can have variables for replacement denoted by `${key_name}`. A payload_context is passed to replace all variables in the payload. As the most common use will be to send a message with a Title/Subject and Text/Body these can be optionally passed and will work with the `default_card.json` template. For card info and designs see [here](https://adaptivecards.io/samples/).

#|hide

NOTE: Functions are added to the class using the `@fastcore.utils.patch` decorator with typehint `self:PingMe` to ensure it's added to the class. Also note it uses the comments on the per line of init to generate the docs, this is a feature from nbdev.

In [ ]:
#|export
from pydantic import BaseModel
class Card(BaseModel):
    name: str
    context: dict

In [ ]:
default_card = Card(name=PINGME_ENV_CARD_NAME, context=PINGME_ENV_CARD_CONTEXT)
print(default_card)

In [ ]:
#|export
from pathlib import Path # for type hinting and file checking
import os
from envyaml import EnvYAML # Allows to loads env vars into a yaml file, https://github.com/thesimj/envyaml

class PingMe:
    """
    PingMe class which notifies via either a webhook or email
    """
    def __init__(self,
                 card: Card, # Card object
                 card_dir: Path, # Directory to look for cards in
                 card_ext: str=".yaml"): # Extension of card file
        self.card:Card = card
        self.card_path:Path = os.path.join(card_dir, f"{self.card.name+card_ext}") # built here to limit user options on api

        if not Path(self.card_path).is_file():
            raise ValueError(f"Payload file does not exist {self}")

        # Resolve config variables from ENV vars
        config:dict = EnvYAML(self.card_path, strict=False).export()
        self.email:dict = config["config"]["email"]
        self.webhook:dict = config["config"]["webhook"]
        self.logfile:dict = config["config"]["logfile"]
        # Resolve title and text from card.context
        self.title:str = self.card.context.get("title", "")
        self.text:str = self.card.context.get("text", "")
        # Resolve payload variables from card.context
        self.payload:json = resolved_payload(config["payload"]["value"], card.context)

    def __str__(self) -> str:
        return (
        f"""PingMe object with:
    card: {self.card}
    card_path: {self.card_path}
    payload:
{self.payload}"""
     )
    def __repr__(self) -> str:
        return self.__str__()

Functionality to resolve the payload, which means changing the variables and returning it as a `str` instead of a `json` object.

In [ ]:
#|export
import re # regular expression for parsing
import json # to manage json payloads
@staticmethod
def resolved_payload(payload:json, context:dict) -> json:
    """
    Resolves the payload by substituting variables in the `payload` with values from the `context` and ensures all variables are accounted for
    """
    if payload is None:
        # Ensure there is a payload
        raise ValueError("Payload is None")
    str_temp = json.dumps(payload) # convert payload to string
    for key in context.keys():
        # Substitute all variables in payload with values from payload_context, it can also be set up that their are no variables in the payload
        str_temp = str_temp.replace("${"+key+"}", context[key])
    if re.search("\${.*}", str_temp):
        # Check if there are any variables left, this is not allowed
        raise ValueError("Unresolved variables in payload")
    return json.loads(str_temp)

Check the notification object loading with defaults and the contents of it

In [ ]:
notification = PingMe(card=default_card, card_dir=PINGME_ENV_CARD_DIR, card_ext=PINGME_ENV_CARD_EXT)
notification

Functionality to send the resolved_payload to a webhook, its split from the class method for the sake of testing.

In [ ]:
#|export
import requests # to send requests to webhooks
@staticmethod
def send_to_webhook(url:str, payload:json, header:json={'Content-Type': 'application/json'}) -> dict:
    if url is None:
        raise Exception("Webhook URL not set")
    # Send message to webhook
    try:
        response = requests.post(url, data=payload, headers=header)
    except Exception as e:
        raise Exception(f"Error sending message to webhook: {e}")
    return {"status_code": response.status_code, "response": response.text}

In [ ]:
# Test it fails with a bad url
with fastcore.test.ExceptionExpected(): send_to_webhook("https://badhost", notification.payload)

In [ ]:
#|export
@patch
def send_webhook(self:PingMe) -> dict:
    return send_to_webhook(self.webhook["url"], self.payload)

Basic tests

If there's an error with this please check if you have a valid `WEBHOOK_URL` in the config.

In [ ]:
# Note this test will send a request out
if SEND_WEBHOOK:
    fastcore.test.test_eq({"status_code": 200, "response": '1'}, notification.send_webhook())

Functionality to send the resolved_payload to an e-mail address

In [ ]:
#|export
import smtplib # to send emails
import email.mime.text # to format emails

@staticmethod
def send_to_email(payload:json, subject:str, from_:str, to:str, host:str, port:int=25, user=None, password=None) -> dict:
    email_status = False
    html_content = f"""
    <html>
    <head>
        <meta http-equiv="Content-Type" content="text/html; charset=us-ascii">
        <script type="application/adaptivecard+json">
            {payload}
        </script>
    </head>
        <body>
            This is a sample body
        </body>
    </html>
    """
    msg = email.mime.text.MIMEText(html_content, 'html')
    msg['Subject'] = subject
    msg['From'] = from_
    msg['To'] = to
    email_connection = smtplib.SMTP(host, port)
    try:
        email_connection.ehlo()
        email_connection.starttls()
        email_connection.ehlo()
        email_connection.login(user, password)
        email_connection.sendmail(from_, to, msg.as_string())
        email_status = True
    finally:
        email_connection.quit()
        return {"status_code":200, "response":email_status}


In [ ]:
# Test if not provided a proper smtp host
with fastcore.test.ExceptionExpected(): send_to_email(
    notification.payload,
    notification.title,
    notification.email["from"],
    notification.email["to"],
    "https://badhost",
    notification.email["smtp"]["port"],
    notification.email["smtp"]["user"],
    notification.email["smtp"]["password"])

In [ ]:
#|export
@patch
def send_email(self:PingMe) -> dict:
    return send_to_email(
        self.payload,
        self.title,
        self.email["from"],
        self.email["to"],
        self.email["smtp"]["host"],
        self.email["smtp"]["port"],
        self.email["smtp"]["user"],
        self.email["smtp"]["password"])

If there's error's here please check the type of error. ConnectionRefusedError indicates there's issues with connecting to the SMTP email server.

In [ ]:
if SEND_EMAIL:
    fastcore.test.test_eq({"status_code": 200, "response": True}, notification.send_email())

Basic test

!|hide
The `send_to_logfile` function right now only sends a title and text, along with a timestamp. This is based off the default card. Some better solution should be found for this in the future. The goal is to not send raw json to the log file as it wont be readable. Perhaps storing each file in a folder and then logging the date and link to file?

In [ ]:
#|export
import datetime # to get current date and time which is used in logging

@staticmethod
def send_to_logfile(logfile:str, title:str, text:str) -> dict:
    """
    Send message to logfile
    TODO: The log file only lods title and text right now (not payload), while payload can be included it is not good for parsing. Need to think of a solution for this. Could be to save each payload as a seperate file and the log is a list of files.
    """
    if logfile is None:
        raise Exception("Log file not set")
    with open(logfile, "a") as f:
        # Write the current time
        f.write(f"{datetime.datetime.now()}\t{title}\t{text}\n")
        f.write("\n")
    return {"status_code":200, "response":True}

In [ ]:
#|export
@patch
def send_logfile(self:PingMe) -> dict:
    return send_to_logfile(self.logfile["path"], self.title, self.text)

In [ ]:
print(notification.logfile["path"])
if SEND_LOGFILE:
    notification.send_logfile()

In [ ]:
# Test if not provided a proper log file
with fastcore.test.ExceptionExpected(): send_to_log_file(None, notification.title, notification.text)

In [ ]:
#|export
# Make a CLI function using `call_parse` to handle arguments
from sys import stderr
from fastcore.script import call_parse
import distutils.util # to convert string to bool
import json
import sys
# Ensure settings.ini contains `console_scripts = pingme=pingme.pingme:cli`, this makes the call as `pingme` and calls the cli function found in package pingme.pingme
@call_parse # https://fastcore.fast.ai/script.html#example
def cli(#TODO: fix params
    config_file:str,
    context:str,
    send_type:str,
    card_name:str = "default", # Name of the card which matches a card found in $PROJECTDIR/card_dir/
    card_dir:str = "./cards/",
    card_ext:str = ".yaml"
    ):
    """
    The Command Line Interface (CLI) for the pingme package. This is the main entry point for the package. Currently only allows for a config file to be provided, but can be extended to allow for args. Command line passing is handled with @call_parse decorator.
    """
    set_env_variables(config_file)
    context = json.loads(context)
    card = Card(name=card_name, context=context)
    pingme = PingMe(card=card, card_dir=card_dir, card_ext=card_ext)

    if send_type == "webhook":
        pingme.send_webhook()
        print("Sent to webhook", file=sys.stderr)
    if send_type == "email":
        pingme.send_email()
        print("Sent to email", file=sys.stderr)
    if send_type == "logfile":
        pingme.send_log_file()
        print("Sent to logfile", file=sys.stderr)

    return True

Test CLI by calling it as a function, should also export and run from command line to check as well.

In [ ]:
cli(config_file=os.environ["PINGME_CONFIG_PATH"], context='{"title":"Test", "text":"This is a test"}', send_type="webhook")

#|hide

[Add in-notebook export cell](https://nbdev.fast.ai/tutorial.html#add-in-notebook-export-cell), which generates the .pymodules. Recommended to run `nbdev_prepare` to export, test and clean. Note this will run on all notebooks so they all have to be buildable.

In [ ]:
#| hide
from nbdev import nbdev_export
nbdev_export()